<a href="https://colab.research.google.com/github/anuragbantu/DL-Assignment-2/blob/main/Assignment2_partB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Imports and Setup

Import all required libraries for data manipulation, model building, and training.


In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2. Loading the Efficientnet model with pretrained weights

In [3]:
from torchvision.models import efficientnet_v2_s

model = efficientnet_v2_s(pretrained=True)



/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth
100%|██████████| 82.7M/82.7M [00:00<00:00, 121MB/s]


## 3. Data Preprocessing

Define transform for data preprocessing. Resize input images to 224 x 224 since this is the size expected by the EfficientNet model.

In [5]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),   # match expected input size
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  # ImageNet mean
                         [0.229, 0.224, 0.225])  # ImageNet std
])


## 4. Data Loading and Train/Validation Split

Specify the dataset paths and load the image dataset using `ImageFolder`.

In [4]:
#if running on google colab, use this cell

import os
import zipfile

zip_path = '/content/drive/MyDrive/nature_12K.zip'

extract_path = '/content/iNaturalist'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted to:", extract_path)

Dataset extracted to: /content/iNaturalist


In [6]:
#load the data
train_dir = '/content/iNaturalist/inaturalist_12K/train'
test_dir = '/content/iNaturalist/inaturalist_12K/val'

train_dataset = ImageFolder(root=train_dir, transform=transform)
test_dataset = ImageFolder(root=test_dir, transform=transform)


## 5. Model Modification

Adapt the pre-trained EfficientNet model to the iNaturalist dataset by modifying the final classifier layer for 10 classes.

In [7]:
# Modify final layer to match number of classes
num_classes = len(train_dataset.classes)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)


In [8]:
# Load the full training dataset
full_train_dataset = ImageFolder(root=train_dir, transform=transform)

# Calculate split sizes
val_size = int(0.2 * len(full_train_dataset))
train_size = len(full_train_dataset) - val_size

# Split the dataset
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

# Loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

# Load the test set
test_dataset = ImageFolder(root=test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


## 6. Freezing Pretrained Layers

Freeze early convolutional blocks to retain pre-trained features while allowing later blocks and classifier layers to fine-tune on the new dataset.

In [10]:
for name, child in model.features.named_children():
    if int(name) < 4:  # Freeze blocks 0 to 3
        for param in child.parameters():
            param.requires_grad = False
    else:
        for param in child.parameters():
            param.requires_grad = True

for param in model.classifier.parameters():
    param.requires_grad = True


In [11]:
import torch

#criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


## 7. Define Loss Function and Optimizer

Use cross-entropy loss for classification and Adam optimizer for training only the unfrozen parameters.

In [12]:
#for freezing use this
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)


In [13]:
import copy

# Save the initial state of the frozen layers (just one block to check)
initial_weights = copy.deepcopy(model.features[0][0].weight.data.clone())


## 8. Training and Validation Loop

Train the model for multiple epochs while tracking loss and accuracy on both training and validation sets.

In [14]:
#Training loop
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (preds == labels).sum().item()

    train_acc = 100 * correct_train / total_train
    train_loss = running_loss / len(train_loader)

    # --- Validation ---
    model.eval()
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
            val_outputs = model(val_inputs)
            _, val_preds = torch.max(val_outputs, 1)
            total_val += val_labels.size(0)
            correct_val += (val_preds == val_labels).sum().item()

    val_acc = 100 * correct_val / total_val

    print(f"Epoch {epoch+1}/{epochs}")
    print(f"  Training Loss: {train_loss:.4f} | Training Accuracy: {train_acc:.2f}%")
    print(f"  Validation Accuracy: {val_acc:.2f}%\n")

Epoch 1/5
  Training Loss: 1.0109 | Training Accuracy: 69.78%
  Validation Accuracy: 86.24%

Epoch 2/5
  Training Loss: 0.3818 | Training Accuracy: 88.58%
  Validation Accuracy: 86.29%

Epoch 3/5
  Training Loss: 0.2027 | Training Accuracy: 94.00%
  Validation Accuracy: 86.64%

Epoch 4/5
  Training Loss: 0.1249 | Training Accuracy: 96.20%
  Validation Accuracy: 87.44%

Epoch 5/5
  Training Loss: 0.0922 | Training Accuracy: 97.31%
  Validation Accuracy: 86.79%



In [15]:
updated_weights = model.features[0][0].weight.data
# Compare
if torch.equal(initial_weights, updated_weights):
    print("Frozen layer weights have not changed.")
else:
    print("Frozen layer weights have changed")
# Compare

Frozen layer weights have not changed.


## 9. Final Evaluation on Test Set

Evaluate the trained model on the held-out test dataset and report test accuracy.

In [17]:
# --- Evaluate on test set ---
model.eval()
correct_test = 0
total_test = 0

with torch.no_grad():
    for test_inputs, test_labels in test_loader:
        test_inputs, test_labels = test_inputs.to(device), test_labels.to(device)
        test_outputs = model(test_inputs)
        _, test_preds = torch.max(test_outputs, 1)
        total_test += test_labels.size(0)
        correct_test += (test_preds == test_labels).sum().item()

test_accuracy = 100 * correct_test / total_test
print(f"Test Accuracy: {test_accuracy:.2f}%")

Test Accuracy: 85.45%


In [18]:
#for epochs = 5
#without freezing test acc = 84.85
#with freezing all layers except last = 70.1
#with freezing upto 4 layers = 85.45
#only pre trained model without finetuning on naturalist = 13.00